In [ ]:
import os, sys
sys.path.append(os.path.abspath("../"))

from math import ceil
import pandas as pd
from shapely.wkt import loads as gload
from shapely.geometry import Point, LineString, Polygon, mapping
import networkx as nx
import tqdm

%load_ext autoreload
%autoreload 2
from config import defaults
from workplaces.workplace import WorkPlace
from converters.converter_for_mongo import convert_mos_list_to_df
from converters.converter_for_mongo import convert_flowsforecast_to_df
from calculators.forecast_calculator import Forecast
from calculators.TR_calculator import TR_calculator

from utils import first_or_none

In [ ]:
# Рабочая область
wp = WorkPlace(name='rounet_loko',
               config=defaults['db_configs']['mongo'])
# БД
db = wp.db.db

In [ ]:
# Создание ТР из Мо
municipalities_df = convert_mos_list_to_df([x for x in db['mun_soc'].find()])
tr_calc = TR_calculator(municipalities_df)

In [ ]:
def binding_plo_tr(point: Point, trs_dict: dict):
    # id для  ТР != РФ
    tr = 999999
    for tr_id, tr_info in trs_dict.items():
        if tr_info['geometry'].intersects(point):
            tr = tr_id
    return tr

# Уникальный список остановок из потоков
flows_df = convert_flowsforecast_to_df([x for x in db['flows'].find()])

plos_from_flows = pd.unique(flows_df[['plo_source',
                                    'plo_target']].values.ravel('K'))

plos_express_info = [x for x in db['plos_express_info'].find()]
plos_express_info_dict = {x['express_code']: {'geometry':
                                              gload(x['geometry_wkt']),
                                              'tr': binding_plo_tr(gload(x['geometry_wkt']),
                                                                   tr_calc.trs)}
                         for x in plos_express_info
                         }

In [ ]:
# Расчитанные на месяц составности
calc_cars = [x for x in db['calc_cars'].find()]

calc_cars_dict = {}

for nit_car_info in calc_cars:
    nit_id = first_or_none(list(nit_car_info.keys()),
                lambda x: x!='_id')
    info = nit_car_info[nit_id]
    calc_cars_dict[int(nit_id)] = info

In [ ]:
# Граф экспресс
EG = nx.MultiGraph()

for edge_info in db['expressg'].find():
    edge_id = first_or_none(list(edge_info.keys()),
            lambda x: x!='_id')
    edge = edge_info[edge_id]
    
    EG.add_edge(edge[0],edge[1],length=edge[2]['length'])

In [ ]:
# Предрасчитанное время следования в минутах
timatables_base_list = [x for x in db['timatables_base'].find()]

timatables_base_dict = {}

for tt_b in timatables_base_list:
    nit_id = first_or_none(list(tt_b.keys()),
                lambda x: x!='_id')
    tt_info = tt_b[nit_id]
    timatables_base_dict[nit_id] = {
        'express_path':tt_info['express_path'],
        'full_minutes_list':tt_info['full_minutes_list']
    }

In [ ]:
# Предрасчитанные расстояния plo-plo
calc_dist_df = pd.DataFrame([x for x in db['eg_calc_flows_plo_plo'].find()])

calc_dist_df['plo_source'] = [str(int(x.InitPassStationExpress)) for x in calc_dist_df.itertuples()]
calc_dist_df['plo_target'] = [str(int(x.FinPassStationExpress)) for x in calc_dist_df.itertuples()]

eg_dict = {x.plo_source + ' ' + x.plo_target: x.eg_dist for x in calc_dist_df.itertuples()}

In [ ]:
uniq_corrs_month = pd.DataFrame([x for x in db['flows_month_format'].find()])

In [ ]:
uniq_corrs_month['init_is_info'] = [1 if x.InitPassStationExpress in plos_express_info_dict.keys() else 0
                             for x in uniq_corrs_month.itertuples()]

uniq_corrs_month['fin_is_info'] = [1 if x.FinPassStationExpress in plos_express_info_dict.keys() else 0
                             for x in uniq_corrs_month.itertuples()]

In [ ]:
report = uniq_corrs_month[uniq_corrs_month['init_is_info']==1]
report = report[report['fin_is_info']==1]
report.columns

In [ ]:
report['tr_source'] = [plos_express_info_dict[x.InitPassStationExpress]['tr'] 
                       for x in report.itertuples()]
report['tr_target'] = [plos_express_info_dict[x.FinPassStationExpress]['tr'] 
                       for x in report.itertuples()]

In [ ]:
report['tr_source_name'] = [tr_calc.trs[x.tr_source]['tr_name']
                                         if x.tr_source in tr_calc.trs.keys()
                                         else 'Мир'
                                     for x in report.itertuples()]
report['tr_target_name'] = [tr_calc.trs[x.tr_target]['tr_name']
                                         if x.tr_target in tr_calc.trs.keys()
                                         else 'Мир'
                                     for x in report.itertuples()]

In [ ]:
base_year = 2019
forecast_year = 2035
report['base_year'] = [base_year for x in report.itertuples()]
report['forecast_year'] = [forecast_year for x in report.itertuples()]

In [ ]:
report['base_year_pop_tr_source'] = [tr_calc.trs[x.tr_source]['POP'][base_year]
                                         if x.tr_source in tr_calc.trs.keys()
                                         else 1
                                     for x in report.itertuples()]

report['base_year_income_tr_source'] = [tr_calc.trs[x.tr_source]['INCOME'][base_year]
                                         if x.tr_source in tr_calc.trs.keys()
                                         else 1
                                     for x in report.itertuples()]

report['forecast_year_pop_tr_source'] = [tr_calc.trs[x.tr_source]['POP'][forecast_year]
                                         if x.tr_source in tr_calc.trs.keys()
                                         else 1
                                     for x in report.itertuples()]

report['forecast_year_income_tr_source'] = [tr_calc.trs[x.tr_source]['INCOME'][forecast_year]
                                         if x.tr_source in tr_calc.trs.keys()
                                         else 1
                                     for x in report.itertuples()]
###

report['base_year_pop_tr_target'] = [tr_calc.trs[x.tr_target]['POP'][base_year]
                                         if x.tr_target in tr_calc.trs.keys()
                                         else 1
                                     for x in report.itertuples()]

report['base_year_income_tr_target'] = [tr_calc.trs[x.tr_target]['INCOME'][base_year]
                                         if x.tr_target in tr_calc.trs.keys()
                                         else 1
                                     for x in report.itertuples()]

report['forecast_year_pop_tr_target'] = [tr_calc.trs[x.tr_target]['POP'][forecast_year]
                                         if x.tr_target in tr_calc.trs.keys()
                                         else 1
                                     for x in report.itertuples()]

report['forecast_year_income_tr_target'] = [tr_calc.trs[x.tr_target]['INCOME'][forecast_year]
                                         if x.tr_target in tr_calc.trs.keys()
                                         else 1
                                     for x in report.itertuples()]

In [ ]:
def get_cars_count(calc_cars_dict, nit_id, month, car_type):
    try:
        res = calc_cars_dict[nit_id][str(month)][car_type]
    except:
        try:
            counter = 0

            for m, m_info in calc_cars_dict[nit_id].items():

                counter+=m_info[car_type]
            res = ceil(counter/len(list(calc_cars_dict[nit_id])))
            
        except:
#             print('00')
            res = 0
    return res
        

In [ ]:
report['coupe_count'] = [get_cars_count(calc_cars_dict,
                                       x.possible_nits_id,
                                       str(x.Month),
                                       'К')
                        for x in report.itertuples()]
report['luxe_count'] = [get_cars_count(calc_cars_dict,
                                       x.possible_nits_id,
                                       str(x.Month),
                                       'Л')
                        for x in report.itertuples()]
report['luxe_soft_count'] = [get_cars_count(calc_cars_dict,
                                       x.possible_nits_id,
                                       str(x.Month),
                                       'М')
                        for x in report.itertuples()]
report['common_count'] = [get_cars_count(calc_cars_dict,
                                       x.possible_nits_id,
                                       str(x.Month),
                                       'О')
                        for x in report.itertuples()]
report['econom_count'] = [get_cars_count(calc_cars_dict,
                                       x.possible_nits_id,
                                       str(x.Month),
                                       'П')
                        for x in report.itertuples()]
report['seat_count'] = [get_cars_count(calc_cars_dict,
                                       x.possible_nits_id,
                                       str(x.Month),
                                       'С')
                        for x in report.itertuples()]

In [ ]:


report['eg_dist'] = [eg_dict[str(int(x.InitPassStationExpress)) + ' ' + str(int(x.FinPassStationExpress))]
                        if str(int(x.InitPassStationExpress)) + ' ' + str(int(x.FinPassStationExpress)) in eg_dict.keys()
                        else 0
                    for x in report.itertuples()]

In [ ]:
rec_rep = report[report['eg_dist']>0]

rec_rep.flows_coupe.sum()+rec_rep.flows_luxe.sum()+rec_rep.flows_luxe_soft.sum()\
+rec_rep.flows_common.sum()+rec_rep.flows_econom.sum()+rec_rep.flows_seat.sum()

In [ ]:
tt_report_data = report[['possible_nits_id','InitPassStationExpress','FinPassStationExpress','eg_dist']].drop_duplicates()

times_list = []
err_count = 0
for x in  tt_report_data.itertuples():
    if str(x.possible_nits_id) in timatables_base_dict.keys():
        route_info = timatables_base_dict[str(x.possible_nits_id)]
        plo_source = str(int(x.InitPassStationExpress))
        plo_target = str(int(x.FinPassStationExpress))
        dist = x.eg_dist
        
        try:
            plo_source_index = route_info['express_path'].index(plo_source)
            plo_target_index = route_info['express_path'].index(plo_target)
            res = route_info['full_minutes_list'][plo_target_index] - route_info['full_minutes_list'][plo_source_index]
        except:
            res = 0
    else:
        res = 0
    times_list.append(res)
    
tt_report_data['time_minute'] = times_list

tt_report_dict = {
    str(x.possible_nits_id) + ' ' + str(x.InitPassStationExpress) +' '+ str(x.FinPassStationExpress):
    x.time_minute
    for x in tt_report_data.itertuples()
}

In [ ]:
report['time_from_tt_base'] = [tt_report_dict[str(x.possible_nits_id)\
                                              + ' ' + str(x.InitPassStationExpress)\
                                              +' '+ str(x.FinPassStationExpress)]
                              for x in report.itertuples()
                              
                              ]

In [ ]:
rec_rep = report[report['time_from_tt_base']>0]

rec_rep.flows_coupe.sum()+rec_rep.flows_luxe.sum()+rec_rep.flows_luxe_soft.sum()\
+rec_rep.flows_common.sum()+rec_rep.flows_econom.sum()+rec_rep.flows_seat.sum()

In [ ]:
rec_rep['speed'] = [x.eg_dist/(x.time_from_tt_base/60) for x in rec_rep.itertuples()]

In [ ]:
rec_rep = rec_rep[rec_rep['speed']>10]

In [ ]:
len(rec_rep)

In [ ]:
rec_rep.columns

In [ ]:
uniq_source_target_corrs = rec_rep[['tr_source','tr_target','Month']].drop_duplicates()
uniq_source_target_corrs['corrs_id'] = [i for i in range(len(uniq_source_target_corrs))]
uniq_source_target_corrs_dict = {str(x.tr_source)+' '+ str(x.tr_target) + ' ' + str(x.Month): x.corrs_id
                                for x in uniq_source_target_corrs.itertuples()
                                }


In [ ]:
rec_rep['corrs_id'] = [uniq_source_target_corrs_dict[str(x.tr_source)+' '+ str(x.tr_target) + ' ' + str(x.Month)]
                      for x in rec_rep.itertuples()]

In [ ]:
rec_rep['revenue_coupe'] = rec_rep['prices_coupe'].values
rec_rep['revenue_luxe'] = rec_rep['prices_luxe'].values
rec_rep['revenue_luxe_soft'] = rec_rep['prices_luxe_soft'].values
rec_rep['revenue_common'] = rec_rep['prices_common'].values
rec_rep['revenue_econom'] = rec_rep['prices_econom'].values
rec_rep['revenue_seat'] = rec_rep['prices_seat'].values

In [ ]:
rec_rep['plo_source_name'] = rec_rep['InitPassStationName'].values
rec_rep['plo_target_name'] = rec_rep['FinPassStationName'].values
rec_rep['month'] = rec_rep['Month'].values
rec_rep['dist'] = rec_rep['eg_dist'].values
rec_rep['time_per_minute'] = rec_rep['time_from_tt_base'].values
rec_rep['route_speed'] = rec_rep['speed'].values

rec_rep['forecast_year_income_tr_source'] = rec_rep['forecast_year_pop_income_source'].values

In [ ]:
report_res = rec_rep[[
    'corrs_id',
    'tr_source_name',
    'tr_target_name',
    'month',
    'plo_source_name',
    'plo_target_name',
    'dist',
    'time_per_minute',
    'route_speed',
    'flows_coupe',
    'flows_luxe',
    'flows_luxe_soft',
    'flows_common',
    'flows_econom',
    'flows_seat',
    'coupe_count',
    'luxe_count',
    'luxe_soft_count',
    'common_count',
    'econom_count',
    'seat_count',
    'revenue_coupe',
    'revenue_luxe',
    'revenue_luxe_soft',
    'revenue_common',
    'revenue_econom',
    'revenue_seat',
    'base_year',
    'forecast_year',
    'base_year_pop_tr_source',
    'base_year_income_tr_source',
    'forecast_year_pop_tr_source',
    'forecast_year_income_tr_source',
    'base_year_pop_tr_target',
    'base_year_income_tr_target',
    'forecast_year_pop_tr_target',
    'forecast_year_income_tr_target'
    
    
]]

In [ ]:
report_res.to_csv('data_switch_model.csv')